# Python bootcamp

This bootcamp will be teaching python through a just-in-time programming approach. That is, instead of walking you through fundamentals step by step, we will jump right into analyzing code and teaching you the tools necessary to understand and modify that code as we go. You can find a more systematic review of python fundamentals in the subfolders of this repository, with the following organization:
- ``0_getting_started`` contains information on what python is and its current popularity and use.
- ``1_fundamentals`` gives an intro to datatypes, the building blocks of any programming language.
- ``2_functions`` introduces you to functions and scope in python.
- ``3_classes`` gives a basic introduction to python classes and a more advanced example of an OLS class.
- ``5_list_comprehensions`` introduces list comprehensions, an efficient method of creating iterable objects.

This notebook starts with an abitious goal: the creation of a python ``class`` object (don't worry, we'll tell you what this is) that runs OLS on data. To do this, we will introduce concepts like importing packages, defining functions, and manipulating numpy matrices, then organize all of these into a `class` object.  Specifically our goals are going to be:
- Make a linear projection function with b0, b1, x as input, y as output
- Write a data generating function
- Give a brief explanation of the scipy.optimize.minimize function
- Minimize the squared errors to estimate b0 and b1
- Create a class that implements the same minimization, 
  that takes data in instantiation, and has an 'estimate' method.

## Package imports

We will be using object types and methods from a couple of different packages in python. These packages must first be installed in the environment you are working in. For the datahub environment we are using for this bootcamp, the necessary packages are already installed. If you want to work on your own computer, you will need to install these using either ``pip``, python's native installer, or preferably the package manager Anaconda. Lucy will go over installation using Anaconda on Friday. For now, note that python packages are importing using the ``import`` command, as below.

In [176]:
# for later
import numpy as np
from scipy.stats import distributions as iid
from scipy.stats import rv_continuous

## Functions: Building blocks of an OLS class
We are going to start by writing functions for the main actions of an OLS class and for generating the simmulated data we will use to test our OLS functions. An OLS estimator must do two things; define a linear model and (to start) minimize the sum of squared errors. We can summarize these activities as:

1. Linear projection: Predict "y" given a set of betas and data X
    - inputs: b0, b1, x
    - outputs: y hat
2. Define the data
    - Inputs: N, true betas
    - Outputs: y, X matrices
3. Minimizer function: minimizes the squared distance between the linear projection and y
    - inputs: A function to minimize (SSE)
    - output: betas that minimize that function


<br><br><br><br>

### 1. Linear projection
Suppose we have a vector X that is N by 2, where the first column is a column of ones, and a vector of betas: b = [b0, b1]. The projection matrix, or the matrix that predicts y, is given by $Xb$.

#### Aside 1: Matrix algebra and some helpful numpy functions.

``numpy`` is a python package that contains number generators, its own matrix objects and methods, and more! Plus it's blazing fast. The building blocks of numpy are called numpy arrays, and they can have as many dimentions as you want. The next block of code shows you how to create some numpy arrays.

Note that we have renamed `numpy` to `np` when importing into this notebook.

In [223]:
m1 = np.array([1, 2, 3]) # shape (3,)
m2 = np.array([[1, 2, 3]]) # shape (1,3)
m3 = np.array([[1, 2, 3], # shape (2,3)
               [4, 5, 6]])
m4 = np.array([[2], [2], [2]]) # shape (3,1)
# We can inspect an array's shape
print(f"My matrix shapes are\n m1: {m1.shape}\n m2: {m2.shape}\n m3: {m3.shape}\n m4: {m4.shape}")

My matrix shapes are
 m1: (3,)
 m2: (1, 3)
 m3: (2, 3)
 m4: (3, 1)


In [178]:
# numpy arrays can be added with +, matrix multiplied with @, or element-wise multiplied with *:
matmult = m3@m4
elementmult = m2*m1
print(f"Matrix multiplying m3, 4 gives:\n {matmult}")
print(f"Element-wise multiplying m1, m3 gives:\n {elementmult}")

Matrix multiplying m3, 4 gives:
 [[12]
 [30]]
Element-wise multiplying m1, m3 gives:
 [[1 4 9]]


In [179]:
# they can also be (element-wise) raised to powers, etc:
print(m1**2)
print(m1*4)

[1 4 9]
[ 4  8 12]


*end of Aside 1* 

---

#### Back to the linear projection function...

Note that the <font color='#ba2121'>text in red</font> documents the function, telling future users (and your future self!) what arguments the function takes in, and what it returns. This "docstring" is optional, but good practice. Tip: being detailed about what data types are acceptable will help you even more!

In [180]:
def linear_projection(X, b):
    '''
    Inputs:
        - X: numpy array of dimensions NxK. The first column is assumed to be a column of ones.
        - b: numpy array of dimensions Kx1.
    Returns:
        - Xb, an Nx1 numpy array
    '''
    # make sure b is the right shape
    b = b.reshape((len(b),1))
    return X@b

We can test this with random X and b values. In python, we can use `numpy.random.rand(a,b)` to sample numbers from a $\text{Uniform}(0,1)$ distribution, and fill a matrix of shape a $ \times $ b (rows $\times$ columns).

In [205]:
# Generate 2 columns of X data, first column with 1's
X1 = np.ones((5,1)); X2 = np.random.rand(5,1)
# Slap columns together in one matrix
X = np.c_[X1, X2]
# Generate 2 random betas
b = np.random.rand(2,1)
# Project!
linear_projection(X,b)

array([[0.27894163],
       [0.37623856],
       [0.44217684],
       [0.50235654],
       [0.47850427]])

 <br> <br> <br> <br>

### 2. Data generating process
We will create data that has N observations and a "true" but noisy relationship between $x$ and $y$. This type of data is used in Monte Carlo simulations to test theory.

#### Aside 2: Random Sampling
- To sample from a $\text{Uniform}(0,1)$ distribution, use `numpy.random.rand(a,b)`.

In [227]:
np.random.rand(5,1)

array([[0.03816418],
       [0.56551672],
       [0.65887349],
       [0.18326635],
       [0.79741131]])

- We can sample from other distributions. For example, we can choose `norm()` from `scipy.stats.distributions` to get standard $\text{Normal}$ distribution.
- To sample from a distribution, use the `scipy.stats.distributions.[DIST].rvs()` function from `scipy.stats.distributions`.
- We renamed `scipy.stats.distributions` as  `iid` when importing.

In [230]:
# Select a distribution
DIST = iid.norm()
# Sample from that distribution
DIST.rvs((3,1))

array([[1.54151495],
       [1.04394466],
       [2.10778515]])

- To create code that is reproduceable, we can "set the randomizer seed." This selects and resets the specific random generation process.
- Every time you set the seed and then resample, you will get the same "random" numbers.

In [216]:
# Sample some "random" numbers
np.random.seed(seed=1234)
DIST.rvs((3,1))

array([[ 0.47143516],
       [-1.19097569],
       [ 1.43270697]])

In [215]:
# Try sampling again!
np.random.seed(seed=1234)
DIST.rvs((3,1))

array([[ 0.47143516],
       [-1.19097569],
       [ 1.43270697]])

- You can use this in Monte Carlo simulations by generating random data using predefined seeds. This means any time someone re-runs your code, they should get the same output.

- <font color='#ba2121'>**Reminder:**</font> `rand()` and `rvs()` take integers (`int`'s) as inputs -- they are the shape of the output matrix.
- In python, there are many data types, like `float` and `bool`.
- You cannot use `float`'s in `rand()` -- see below output.
- see the `1_fundamentals/Day 1.1 - Python Data Types.ipynb` notebook for more details).

In [228]:
np.random.rand(5.0,1)

TypeError: 'float' object cannot be interpreted as an integer

*end of Aside 2* 

---

#### Back to generating random data...
Make a function that generates X and Y data from $\beta$ values we give it.

In [182]:
def dataGenerator(beta, N, seed=1234):
    '''
    Inputs:
        - beta: A Kx1 numpy array. True beta from which to generate data.
        - N: Number of observations the data should have
        - seed: (optional) integer to make the random number generation repeatable.
    Returns:
        - X: A numpy array of random data with shape NxK
        - y: A numpy array generated by Xbeta + e with shape Nx1
    '''
    # Set the seed for generating random numbers
    np.random.seed(seed=seed)
    # create an X vector
    # note I instantiate iid.norm() and call method rvs() in the same step!
    x = iid.norm().rvs((N,1))

    # create a random error
    e = iid.norm().rvs((N,1))
    # add an intercept by horizontally stacking x with an array of ones,
    X = np.c_[np.ones((N,1)), x]
    # make sure beta is the right shape: Kx1
    beta = beta.reshape((beta.shape[0],1))
    # create y
    y = linear_projection(X, beta) + e
    
    return X, y

Test the function: give the function a true $\beta$ vector and number of observations `N`.

In [219]:
# test
beta_true = np.array([1,1])
dataGenerator(beta_true, 5)

(array([[ 1.        ,  0.47143516],
        [ 1.        , -1.19097569],
        [ 1.        ,  1.43270697],
        [ 1.        , -0.3126519 ],
        [ 1.        , -0.72058873]]),
 array([[ 2.3585981 ],
        [ 0.66861272],
        [ 1.79618346],
        [ 0.70304448],
        [-1.96327369]]))

Try again -- will the data be the same?

In [218]:
# test
beta_true = np.array([1,1])
dataGenerator(beta_true, 5)

(array([[ 1.        ,  0.47143516],
        [ 1.        , -1.19097569],
        [ 1.        ,  1.43270697],
        [ 1.        , -0.3126519 ],
        [ 1.        , -0.72058873]]),
 array([[ 2.3585981 ],
        [ 0.66861272],
        [ 1.79618346],
        [ 0.70304448],
        [-1.96327369]]))

**Dimension check:** How many elements are in the output from `dataGenerator()`?

In [234]:
len(dataGenerator(beta_true, 5))

2

**Type check:** What type of object is the output from `dataGenerator()`?

In [233]:
type(dataGenerator(beta_true, 5))

tuple

Now create the data that we will use for the rest of the notebook by assigning it to the variables `X` and `y`. Because the output from `dataGenerator()` is a tuple, we can "unpack" the elements of the tuple directly into other variables using the following synatax:
```python
A = (1, 2, 3)  # this is a tuple of length 3
a, b, c = A    # this unpacks the elements of A into new variables a,b,c
```

In [221]:
# create data
beta_true = np.array([2,8])
N = 100

X, y = dataGenerator(beta_true, N)

# test shapes!
print(f"X is {X.shape[0]} x {X.shape[1]}")
print(f"y is {y.shape[0]} x {y.shape[1]}")

X is 100 x 2
y is 100 x 1


 <br> <br> <br> <br>

### 3. Minimizer function
In order to minimize, we are going to use a minimizer function from ``scipy.optimize``. The documentation for this function can be found [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html), but the key arguments are the following:
* ``fun``: the function to be minimized. This must be a function of only one input; if there are multiple inputs, we will "mask" these using lambda functions. Note that functions can be passed to other functions! Functions are objects just like other python data types, so we can pass them around using their name.
* ``x0``: The start guess for the solution. In the case that the solution has a global minimum (as the least squares problem does) the choice will only affect computation time.

Thus the final syntax is `minimize(fun = function(x), x0 = [start guess])`.

The function returns an instance of the ``OptimizeResult`` class, which has several attributes. The only one we will be interested in for now is ``x``, the solution that solves the minimization.

Let's set up a function that returns the object we want to minimize for OLS: the sum of squared errors. Note that the SSE is given by:

$$SSE = \sum_i (\widehat{y}_i - y_i)^2$$

Implementing this sum of squared errors in code:

In [185]:
def sse(y, X, b):
    '''
    Inputs:
        - y: Numpy array Nx1
        - X: Numpy array NxK
        - b: Numpy array Kx1
    '''
    yhat = linear_projection(X, b)
    sse = np.sum((yhat - y)**2)
    return sse

In [186]:
sse(y,X,beta_true)

96.00922942677013

**Knowledge check:** we're testing the true $\beta$ vector on the `X` and `y` we generated using `dataGenerator()`... if we're using the true $\beta$ vector, shouldn't the error be zero? Why or why not?

To minimize this function, we need to make it a function of just one variable (the variable we want to minimize over). We can do this by masking the other inputs in a lambda function. What's a lambda function?

#### Aside 3: Passing functions and lambda functions
- Functions in python are a type of object (like a `float`, a `tuple`, or an `array`).
- Any object in python can be passed to a function as an argument.
- To minimize a function in python over one variable, we need to pass a one-argument function to `scipy.optimize.minimize()`.
- The general syntax is like this:

In [241]:
from scipy.optimize import minimize

def my_fun(a):
    return a[0]**2 + a[1]**2

minimize(my_fun, x0 = [1,1])
# `x0` tells `minimize()` where to start searching for the minimizing value.

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 2.311471135620994e-16
        x: [-1.075e-08 -1.075e-08]
      nit: 2
      jac: [-6.600e-09 -6.600e-09]
 hess_inv: [[ 7.500e-01 -2.500e-01]
            [-2.500e-01  7.500e-01]]
     nfev: 9
     njev: 3

You can see that `my_fun` is being passed as an argument to `minimize()`.

What if our function has multiple inputs but we only want to minimize over one? We can use a lambda function to create new function, inside the argument. Suppose we have the following function:

In [242]:
def my_fun2(a, b):
    return b*(a[0]**2 + a[1]**2)

In [246]:
minimize(my_fun2, x0 = [1,1])

TypeError: my_fun2() missing 1 required positional argument: 'b'

This produces an error because `minimize` can only take a function of one argument, yet `my_fun2` takes two arguments....

Lambda functions just create a new "anonymous" function (a funtion without a name). The below lambda function is the same as `my_fun()` but does not have an assigned name to it.
```python
lambda a: a[0]**2 + a[1]**2
```

This allows us to create new functions...
```python
my_fun2 = lambda a: my_fun(a)
```

and newer functions that are functions of some but not all of the same arguments:
```python
b = 2
my_fun3 = lambda a: my_fun2(a, b)
```

This allows us to pass `minimize()` a new function that only has one argument.

In [244]:
b = 2
my_fun3 = lambda a: my_fun2(a, b)

We can put our new function directly inside the minimize function

In [245]:
minimize(my_fun3, x0 = [1,1])

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 4.622942271241988e-16
        x: [-1.075e-08 -1.075e-08]
      nit: 2
      jac: [-1.320e-08 -1.320e-08]
 hess_inv: [[ 6.250e-01 -3.750e-01]
            [-3.750e-01  6.250e-01]]
     nfev: 9
     njev: 3

We can also define the lambda function direclty inside the `minimize` function.

In [247]:
minimize(lambda a: my_fun2(a, b), x0 = [1,1])

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 4.622942271241988e-16
        x: [-1.075e-08 -1.075e-08]
      nit: 2
      jac: [-1.320e-08 -1.320e-08]
 hess_inv: [[ 6.250e-01 -3.750e-01]
            [-3.750e-01  6.250e-01]]
     nfev: 9
     njev: 3

*end of Aside 3* 

---

**Back to minimizing our sum of squared errors...**

In [187]:
from scipy.optimize import minimize

In [188]:
# remember the syntax: 
# minimize(fun = function(x), x0 = [start guess])
# the lambda function allows sse to be a function of only x, the other inputs
# come from the variables X and y we already defined.
minimize(lambda x: sse(y, X, x), x0 = [0,0])
# as expected, we get an intercept of around 2 and a slope around 8

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 95.1005620152201
        x: [ 1.947e+00  8.081e+00]
      nit: 6
      jac: [-9.537e-07 -1.907e-06]
 hess_inv: [[ 5.006e-03 -1.771e-04]
            [-1.771e-04  5.043e-03]]
     nfev: 27
     njev: 9

In [189]:
# let's do it again with a higher N, letting the LLN work for us!
X, y = dataGenerator(beta_true, 10000)
minimize(lambda x: sse(y, X, x), x0 = [0,0])
# now it's even more accurate!

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 9881.846756005334
        x: [ 1.998e+00  7.994e+00]
      nit: 7
      jac: [ 0.000e+00  0.000e+00]
 hess_inv: [[ 5.012e-05 -2.255e-06]
            [-2.255e-06  5.045e-05]]
     nfev: 48
     njev: 16

<br><br><br><br>

## Creating an OLS class
Now let's take our functions and organize them into an OLS class. What is a python class? 

python classes are objects that have attributes and methods. These objects are abstract in the sense that a class in the abstract is used to "instantiate" specific "instances" of the class. For example, here we will create a class called OLS, and will create several instantiations of specific OLS models. That is, the OLS class might have attributes like X and y data, but an OLS class _instance_ will have _specific_ data for X and y.

To define a class, we use ``class ClassName:``, followed by indented lines (convention is to name classes with upper case).

Classes have 3 components:
* **The constructor** this is a method (think: function) that creates an _instance_ of the class. This component must be present in all classes, as it is the engine that creates the object. It looks like ``def __init__(self):``
* **Attributes** These are attributes that all instances of a class have. They can be anything, or not exist at all. For example, consider a Student class that stores information for a student database. In that case, we might want students to have attributes like a student ID, gender, age, etc. These can vary by instance of the Student class, but all instances of ``Student`` have one. If we imagine an alternative class called `Alien()`, attributes might include home planet and and number of limbs.
* **Methods** Methods are functions that belong to a class. They may return output or not. For example, we may want to write a method that calculates a student's GPA given a series of numeric grades. In the case of OLS, out main method will be to estimate beta.

The simplest class is made up of only a constructor, and the simplest constructor doesn't do anything except create an instance of the class. This looks like:

In [190]:
class OLS:
    # constructor
    def __init__(self):
        pass

Now to create an _instance_ of the class, we do the following:

In [191]:
myOLSmodel = OLS()
myOLSmodel

However, this OLS model is not very interesting. For example, we might think that each OLS model might have data associated with it. Therefore, we can require that the instantiation receive X and y data, and create attributes from these data. Attributes "belong" to the class, and can be accessed with the syntax ``classInstance.attribute``.

In [192]:
class OLS:
    # constructor
    def __init__(self, X, y):
        # define attributes
        self.X = X
        self.y = y

In [193]:
y_test = np.ones((5,1))
x_test = np.ones((5,2))*3

myOLS = OLS(x_test, y_test)
myOLS.y # note that within OLS(), the argument is called y!

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.]])

Now that our OLS class has attributes, we want it to calculate something! So we are going to give it **methods**, which are functions that belong to a class. For example, let's add our linear_projection function to this class as a method. In order to do this, we define the function within the class body, and we add another special argument to the method ``self``. By adding self as an argument, we will have access to all the attributes and methods that the class contains! For example:

In [194]:
class OLS:
    # constructor
    def __init__(self, X, y):
        # define attributes
        self.X = X
        self.y = y
    
    # ------- methods --------
    # linear projection
    def linear_projection(self, b):
        b = b.reshape((len(b),1))
        return self.X@b

What do you notice is different about linear_projection the method?
1. It has ``self`` as an argument. This allows it to know about attributes and even other methods contained in the class.
2. I took away X as an argument, instead calling ``self.X`` in the method body. How can this be?? Since X is now an _attribute_ of the class, and the method has the ``self`` argument, ``self.X`` is saying "grab the X that you defined as the class attribute". This way we don't have to constantly be entering our data into all the function calls, because our OLS instance is storing it for us! Here's how we would use this method:

In [195]:
myOLS = OLS(X, y)

# method call with instanceName.method. Only input is b; we don't have to pass "self" to the method.
myOLS.linear_projection(np.array([2,1]))

array([[2.47143516],
       [0.80902431],
       [3.43270697],
       ...,
       [0.94753931],
       [1.5023069 ],
       [1.7439938 ]])

Now let's add our other methods. What other changes do you notice to these expressions?

In [196]:
class OLS:
    # constructor
    def __init__(self, X, y):
        # define attributes
        self.X = X
        self.y = y
    
    # ------- methods --------
    # linear projection
    def linear_projection(self, b):
        b = b.reshape((len(b),1))
        return self.X@b
    
    # SSE
    def sse(self, b): # used to be a f'm of y, X, and b. Now only need b!
        yhat = self.linear_projection(b)
        sse = np.sum((yhat - self.y)**2)
        return sse
    
    # minimize the SSE
    def estimate(self, x0 = [0,0]):
        # default initial guess of [0,0]
        sol = minimize(self.sse, x0 = x0)
        return sol.x

What happened to the arguments in minimize?? Here something cool happens, and it actually starts with the `sse` function. Now that `X` and `y` are attributes of the class, the `self.sse()` method knows what they are thanks to the `self` argument that is implicitly passed into it! Therefore we can call `sse()` as only a function of one argument: `b`. Here's proof:

In [197]:
# instantiate model
model1 = OLS(X, y)

# call sse method
model1.sse(np.array([2,1]))

494408.17393946426

Now that `OLS.sse()` is only a function of one argument, we can omit the arguments altogether in the minimze function, and call it just by its name, `self.sse`. This is an example of passing a function to another function; when there is only 1 argument, we only need its name. And because of the class structure, it already knows that the single argument is what it is minimizing over! Let's test it:

In [198]:
# call the solve_OLS() method
model1.estimate()

array([1.99794054, 7.99362178])

<br><br><br><br>

## Exercises
Try these out for yourself:
1. After estimating $\widehat{\beta}$, add it as an attribute of the OLS class.
2. Estimate the White-robust SEs. There are a few ways to do this; which do you prefer? Why?
     - Estimate and return them with beta in a tuple
     - Estimate them with beta and add as an attribute
     - Write a method that calculates and returns them upon request (nice code will avoid re-estimating the betas each time you do this. How can this be avoided?)
3. Rewrite the estimation in terms of matrix algebra instead of a minimization.

In [199]:
class OLS:
    # constructor
    def __init__(self, X, y):
        # define attributes
        self.X = X
        self.y = y
    
    # ------- methods --------
    # linear projection
    def linear_projection(self, b):
        b = b.reshape((len(b),1))
        return self.X@b
    # SSE
    def sse(self, b): # used to be a f'm of y, X, and b. Now only need b!
        yhat = self.linear_projection(b)
        sse = np.sum((yhat - self.y)**2)
        return sse
    # minimize the SSE
    def estimate(self, x0 = [0,0]):
        # default initial guess of [0,0]
        sol = minimize(self.sse, x0 = x0)
        # Exercise 1
        self.beta = sol.x
        
        return sol.x
        
        
    # Exercise 2
    def whiteSEs(self):
        if not hasattr(self, 'beta'):
            self.estimate()
        e = self.y - self.linear_projection(self.beta)
        emat = np.diag((e**2).squeeze()) # squeeze to make 1D for np.diag() alt: e**2 * np.eye(length(e))
        bread = np.linalg.inv(self.X.T@self.X)
        whiteSE = bread@(self.X.T@emat@self.X)@bread
        return(whiteSE)
    
     # Exercise 3
    def estimate_matAlg(self):
        return np.linalg.inv(self.X.T@self.X)@self.X.T@self.y

<br><br><br><br>

### Test our work
I test the results against the ``statsmodels`` package.

In [200]:
import statsmodels.api as sm

In [201]:
# instantiate
model = OLS(X, y)
# run
model.estimate()

array([1.99794054, 7.99362178])

In [202]:
model.whiteSEs()

array([[ 9.88373836e-05, -1.39779499e-06],
       [-1.39779499e-06,  1.00617947e-04]])

In [66]:
print(f"White SEs: \n{model.whiteSEs()**(1/2)}")
print(f"beta_hat (minimizer): \n {model.beta}")
print(f"beta_hat (matrix algebra): \n{model.estimate_matAlg()}")

White SEs: 
[[0.0101291         nan]
 [       nan 0.01016232]]
beta_hat (minimizer): 
 [2.00444031 7.99636302]
beta_hat (matrix algebra): 
[[2.00444032]
 [7.99636302]]


C:\Users\A\AppData\Local\Temp\ipykernel_22568\774340785.py:1: RuntimeWarning: invalid value encountered in sqrt
  print(f"White SEs: \n{model.whiteSEs()**(1/2)}")


Note that 

In [28]:
robust_ols = sm.OLS(y, X).fit(cov_type='HC0')

In [29]:
print("statsmodels White SEs:\n %s" % robust_ols.cov_params()**(1/2))
print("statsmodels beta_hat:\n%s" % robust_ols.params)

statsmodels White SEs:
 [[0.01003197 0.00160577]
 [0.00160577 0.01030198]]
statsmodels beta_hat:
[1.99789902 8.00919577]
